In [1]:
import pandas as pd
from pythainlp import word_tokenize as tokenize
from pythainlp.util import *
from tqdm.auto import tqdm
import sys
from collections import Counter
sys.path.append('../Cython')
from Method import *
tqdm.pandas()

C:\Users\Alps\anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
df = pd.read_csv('../assets/SplitData/2021_10_24.csv')
df

,title,summary,link,published
0,เปิด 5 ประเด็นจาก 5 เวที GCNT Forum 2021รวมพลั...,เปิด 5 ประเด็นจาก 5 เวที GCNT Forum 2021รวมพลั...,https://thaipublica.org/2021/10/gcnt-forum-202...,2021_10_24
1,PTTGC ทุ่มงบ 1.65 แสนล้าน ตั้งเป้าลดการปล่อยก๊...,PTTGC ทุ่มงบ 1.65 แสนล้าน ประกาศแผนงานสู่เป้าห...,https://thaipublica.org/2021/10/pttgc-together...,2021_10_24
2,ASEAN Roundup เวียดนามเปิดรับวัคซีนพาสปอร์ต 72...,ASEAN Roundup ประจำวันที่ 17-23 ตุลาคม 2564 เว...,https://thaipublica.org/2021/10/asean-weekly-r...,2021_10_24
3,กองทุนน้ำมันเชื้อเพลิงติดกับดัก “ประชานิยม”,กองทุนน้ำมันเชื้อเพลิงติดกับดัก “ประชานิยม” คว...,https://thaipublica.org/2021/10/oil-fuel-fund-...,2021_10_24
4,วิกฤติศรัทธาของชาวฮินดูและไทยพุทธ,การปฏิรูปกิจกรรมพระพุทธศาสนาโดยการเพิกเฉยต่อวั...,https://thaipublica.org/2021/10/mano-laohavani...,2021_10_24
...,...,...,...,...
599,&quot;นิพนธ์&quot; สั่ง ผู้ว่าฯ เตือนประชาชน เ...,"""รมช.นิพนธ์"" กำชับ เจ้าหน้าที่รับมือสถานการณ์น...",https://www.thairath.co.th/news/politic/2226831,2021_10_24
600,รอบรั้วการตลาด : แม็คโครจัด Taste of Australia...,รอบรั้วการตลาด ประจำวันที่ 24 ต.ค. 64 ทีมข่าวไ...,https://www.thairath.co.th/business/market/222...,2021_10_24
601,จับแรงงานพม่า 120 คน ซ่อนตัวในป่าไทรโยค จะไปทำ...,แรงงานเถื่อนทะลัก ที่ จ.กาญจนบุรี ทหารสนธิกำลั...,https://www.thairath.co.th/news/local/central/...,2021_10_24
602,แอนชิลี สวยชนะใจกรรมการ ฟาดมง MUT2021,รอๆ กันมานาน ในที่สุดก็ได้นางงามคนใหม่กันแล้ว ...,https://www.thairath.co.th/entertain/news/2226591,2021_10_24


In [3]:
NEWMM = "newmm"
ATTACUT = "attacut"
list_column = df.columns.values.tolist()
column_name = list_column[1]

def word_break(news,engine):
    segment = tokenize(str(news),keep_whitespace=False,engine=engine)
    return segment

In [4]:
def read_stop_keyword(filename):
    text_file = open(f"../assets/StopKeywordList/{filename}.txt", "r", encoding='utf-8')
    content = text_file.read()
    dict_list = content.split("\n")
    text_file.close()
    return dict_list

In [5]:
newmm_stop_keyword = read_stop_keyword("newmm_stopkeyword_m10_th20")

In [6]:
attacut_stop_keyword = read_stop_keyword("attacut_stopkeyword_m10_th20")

In [7]:
atta_list = []
newmm_list = []

def wordBreak(df):
    l = len(df)
    for i in tqdm(range(l)):
        wb_list = word_break(df.iloc[i][column_name],ATTACUT)
        atta_list.append(wb_list)
        wb_list = word_break(df.iloc[i][column_name],NEWMM)
        newmm_list.append(wb_list)

wordBreak(df)

100%|██████████| 604/604 [00:27<00:00, 22.26it/s]


In [8]:
def LCS(atta_list, newmm_list):
    l = len(atta_list)
    lcs_atta = []
    lcs_newmm = []
    for i in tqdm(range(l-1)):
        w1_atta = atta_list[i]
        len_w1_atta = len(w1_atta)
        w1_newmm = newmm_list[i]
        len_w1_newmm = len(w1_newmm)
        for j in range(i+1,l):
            w2_atta = atta_list[j]
            len_w2_atta = len(w2_atta)
            w2_newmm = newmm_list[j]
            len_w2_newmm = len(w2_newmm)
            lcs_atta_list,maxl_atta = [],0
            lcs_newmm_list,maxl_newmm = [],0
            if i != j:
                lcs_atta_list,maxl_atta = lcs(w1_atta,w2_atta,len_w1_atta,len_w2_atta)
                lcs_newmm_list,maxl_newmm = lcs(w1_newmm,w2_newmm,len_w1_newmm,len_w2_newmm)
                if lcs_atta_list != []:
                    lcs_atta.extend(lcs_atta_list)
                if lcs_newmm_list != []:
                    lcs_newmm.extend(lcs_newmm_list)

    lcs_atta_counter = Counter(lcs_atta)
    lcs_newmm_counter = Counter(lcs_newmm)
    #-------------------------------Remove Stop Keyword---------------------------------------------------
    lcs_atta_counter = {key:value for key,value in lcs_atta_counter.items() if key not in attacut_stop_keyword}
    lcs_newmm_counter = {key:value for key,value in lcs_newmm_counter.items() if key not in newmm_stop_keyword}
    #-----------------------------------------------------------------------------------------------------
    return lcs_atta_counter, lcs_newmm_counter

In [9]:
def LCS_result(data):
    # print("result")
    df_result = pd.DataFrame(data.items(), columns=['LCS', 'round'])
    df_result['total_round'] = df_result['round'].sum()
    df_result.sort_values(['round'], ascending=False, inplace=True, ignore_index=True)
    return df_result

In [10]:
lcs_atta, lcs_newmm = LCS(atta_list, newmm_list)

100%|██████████| 603/603 [00:35<00:00, 16.97it/s]


In [11]:
result_newmm = LCS_result(lcs_newmm)
result_newmm[:20]

,LCS,round,total_round
0,24ต.ค.64,724,5473
1,24ตุลาคม2564,369,5473
2,ในพื้นที่จังหวัด,111,5473
3,(24ต.ค.,87,5473
4,24ต.ค.2564,84,5473
5,"SubmittedonSun,2021-10-",78,5473
6,[…]Thepost“,71,5473
7,ผู้สื่อข่าวรายงานว่า,58,5473
8,โดยสำนักข่าวอินโฟเควสท์(24,54,5473
9,เมื่อวันที่24ตุลาคม2564,52,5473


In [12]:
result_atta = LCS_result(lcs_atta)
result_atta.head(20)

,LCS,round,total_round
0,24ตุลาคม2564,302,8656
1,วันที่24,215,8656
2,24ต.ค.64,127,8656
3,พรรคการเมือง,99,8656
4,พรรคเพื่อไทย,87,8656
5,(24ต.ค.,87,8656
6,ระหว่างวันที่,85,8656
7,"SubmittedonSun,2021-10-",78,8656
8,การทำงาน,62,8656
9,ผู้สื่อข่าวรายงานว่า,61,8656
